# Data cleaning and Exploratory Data Analysis (EDA)

In this notebook, we perform some dataset cleaning and an exploratory data analysis
(EDA).

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('bmh')

In [ ]:
df = pd.read_csv('data/eda/house_prices.csv')
df.head()

In [ ]:
df.info()

Before performing the EDA, let us remove the features with too many missing values (such
as `Alley` and `PoolQC`), plus `Id` (which does not really give any insight) and the
features with max 30% of `NaN` values.

See also [here](https://www.kaggle.com/code/rsiva1104/data-cleaning) for other dataset
cleaning techniques (such as _imputation_).

In [ ]:
# df.count() does not include NaN values
df2 = df[[column for column in df if df[column].count() / len(df) >= 0.3]]
del df2['Id']
print("List of dropped columns:")
for c in df.columns:
    if c not in df2.columns:
        print(c)
print('\n')
df = df2

The dropped features have `NaN` values as most of the houses do not possess them. Now, let's take a look at how the house prices are distributed:

In [ ]:
print(df['SalePrice'].describe())
plt.figure(figsize=(9, 8))
sns.histplot(df['SalePrice'], color='g', bins=100, kde=True, alpha=0.4)

From this plot, we can see that the prices are skewed right and some outliers lie above ~500,000. We will eventually want to get rid of them to get a normal distribution of the variable `SalePrice`.

## Numerical data distribution

In this section, we look at the distribution of the _numerical_ features.

In [ ]:
# list all the data types in the dataset
list(set(df.dtypes.tolist()))

In [ ]:
# only select numerical features
df_num = df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

In [ ]:
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8); # ; avoid having the matplotlib verbose informations

Features such as `1stFlrSF`, `TotalBsmtSF`, `LotFrontage`, `GrLiveArea`... seem to share
a similar distribution to that of `SalePrice`. 

## Feature relationships with `SalePrice`

We can get further insight by trying to find which features are strongly correlated with `SalePrice`. 

In [ ]:
# exclude last column, because it is SalePrice, not a feature
df_num_corr = df_num.corr()['SalePrice'][:-1]
golden_features_list = df_num_corr[abs(df_num_corr) > 0.5].sort_values(ascending=False)
print("There are {} strongly correlated features with SalePrice:\n{}".format(len(golden_features_list), golden_features_list))

This list is incomplete as we know that correlation is affected by outliers. So we could proceed as follow:

- Plot the numerical features and see which ones have very few or explainable outliers
- Remove the outliers from these features and see which one can have a good correlation without their outliers
    
Notice that, correlation by itself does not completely explain relationships within the
data. For example, non-linear relationships cannot be identified by looking at the
correlation value, so we should take the features we excluded from the `golden_features_list` and
plot them to see if they show some kind of patten. 

In [ ]:
for i in range(0, len(df_num.columns), 5):
    sns.pairplot(data=df_num,
                x_vars=df_num.columns[i:i+5],
                y_vars=['SalePrice'])

We can clearly identify some approximately linear relationships with the `SalePrice`. By
looking closely, can see that many features have zero values that may indicate the absence
of such feature in the house. Let us remove these 0 values and repeat the process.

Some plots, such as `GarageCars` -> `SalePrice` or `Fireplaces` -> `SalePrice`, show a
particular pattern with verticals lines roughly meaning that they are discrete variables
with a limited range.

We cannot see any notable pattern involving the features that are
not highly correlated with `SalePrice`.

In [ ]:
import operator

individual_features_df = []
for i in range(0, len(df_num.columns) - 1): # -1 because the last column is SalePrice
    tmpDf = df_num[[df_num.columns[i], 'SalePrice']]
    tmpDf = tmpDf[tmpDf[df_num.columns[i]] != 0]
    individual_features_df.append(tmpDf)

all_correlations = {feature.columns[0]: feature.corr()['SalePrice'][0] for feature in individual_features_df}
all_correlations = sorted(all_correlations.items(), key=operator.itemgetter(1))

By cleaning the dataset we found another strongly correlated feature:

In [ ]:
golden_features_list = [key for key, value in all_correlations if abs(value) >= 0.5]
print("There is {} strongly correlated values with SalePrice:\n{}".format(len(golden_features_list), golden_features_list))

## Feature to feature relationships

In [ ]:
corr = df_num.drop('SalePrice', axis=1).corr() # We already examined SalePrice correlations
plt.figure(figsize=(12, 10))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.4)], 
            cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);

Many features seem to be correlated with each other, but some of them such as
`YearBuild`/`GarageYrBlt` may just indicate a price inflation over the years. As for
`1stFlrSF`/`TotalBsmtSF`, it is normal that the larger is the 1st floor (considering many houses
have only 1 floor), the larger is the basement.

Now, for the ones which are less obvious we can see that:
- There is a strong negative correlation between `BsmtUnfSF` (Unfinished square feet of basement area) and `BsmtFinSF2` (Type 2 finished square feet). There is a definition of unfinished square feet [here](http://www.homeadvisor.com/r/calculating-square-footage/) but as for a house of "Type 2", I can't tell what it really is.
- `HalfBath`/`2ndFlrSF` is interesting and may indicate that people gives an importance
  of not having to rush downstairs in case of urgently having to go to the bathroom.

When building a ML model, correlated features may be combined or selected in order to reduce the number of features.

## Categorical features

Some of the features of our dataset are categorical. To separate the categorical from quantitative features lets refer ourselves to the `data_description.txt` file. According to this file we end up with the folowing columns:

In [ ]:
quantitative_features_list = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']
df_quantitative_values = df[quantitative_features_list]
df_quantitative_values.head()

In [ ]:
# quantitative_features_list[:-1] as the last column is SalePrice and we want to keep it
categorical_features = [a for a in quantitative_features_list[:-1] + df.columns.tolist() if (a not in quantitative_features_list[:-1]) or (a not in df.columns.tolist())]
df_categ = df[categorical_features]
df_categ.head()

In [ ]:
df_not_num = df_categ.select_dtypes(include = ['O'])
print('There is {} non numerical features including:\n{}'.format(len(df_not_num.columns), df_not_num.columns.tolist()))

In [ ]:
plt.figure(figsize = (10, 6))
ax = sns.boxplot(x='BsmtExposure', y='SalePrice', data=df_categ)
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k")
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize = (12, 6))
ax = sns.boxplot(x='SaleCondition', y='SalePrice', data=df_categ)
plt.setp(ax.artists, alpha=.5, linewidth=2, edgecolor="k")
plt.xticks(rotation=45)
plt.show()

And finally lets look at their distribution

In [ ]:
print(df_not_num.columns)

In [ ]:
import math
fig, axes = plt.subplots(math.ceil(len(df_not_num.columns) / 3), 3, figsize=(12, 30))

for i, ax in enumerate(axes.flat):  # Iterate over each subplot
    if i < len(df_not_num.columns):
        # Plot with Seaborn and set rotation for x-axis labels
        sns.countplot(x=df_not_num.columns[i], alpha=0.7, data=df_not_num, ax=ax)
        
        # Rotate x-axis tick labels after ensuring the ticks have been set by countplot
        ax.tick_params(axis='x', rotation=45)

fig.tight_layout()
plt.show()

We can see that some categories are predominant for some features such as `Utilities`, `Heating`, `GarageCond`, `Functional`. These features may not be relevant for our predictive model.